In [1]:
import numpy as np

# Q1

In [15]:
def resolutionSystem1(alpha,beta,delta,gamma,y0,z0,t0,dt,N):
    "Cette fonctipn permet de resoudre le système (1)"
    temps = np.arange(0,N)*dt+t0 # si j'ai bien compris la question (dt=10/N pour 10 années)
    y = [y0]
    z = [z0]
    for i in range(0,N-1):
        y.append(y[i]+dt*y[i]*(alpha-beta*z[i]))
        z.append(z[i]+dt*z[i]*(delta*y[i]-gamma))

    return temps,y,z

# Q2